# Rate My Beer Scraper

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [2]:
# Set up ChromeDriver path
driver_path = "/Users/ramzikattan/Downloads/chromedriver-mac-arm64/chromedriver"
chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"

# Configure Chrome options
chrome_options = Options()
chrome_options.binary_location = chrome_path

# Set up the Chrome WebDriver service
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [ ]:
# Open the webpage
url = "https://www.ratebeer.com/top-beers?time=all"
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Parse the page with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Close the browser after fetching the page
driver.quit()

# Find all <a> tags with the class containing the beer name and link
beer_links = soup.find_all('a', class_="MuiTypography-root")

# Extract the name and URL for each beer
beers = []
for beer in beer_links:
    name = beer.get_text(strip=True)  # Get the text (beer name)
    link = beer['href']  # Get the URL
    full_link = "https://www.ratebeer.com" + link  # Construct full URL
    beers.append({'name': name, 'link': full_link})

# Print the extracted beer names and their URLs
#for beer in beers:
 #   print(f"Beer: {beer['name']} - URL: {beer['link']}")

In [ ]:
# Clean the matrix for only the beer links
cleaned_beers = [beer for beer in beers[28:128] if beer['name']]  # Only keep non-empty names

In [10]:
# Rescrape Using Nico's Links
cleaned_beers = pd.read_csv('beer_url.csv')

# Drop rows with duplicate beer names, keeping the first occurrence
cleaned_beers = cleaned_beers.drop_duplicates(subset=['Beer Name'], keep='first')

# Display the first few rows of the updated DataFrame
cleaned_beers.head()

,Beer Name,Beer Rating,Beer URL
0,Aecht Schlenkerla Rauchbier Urbock,4.0,https://www.ratebeer.com/beer/aecht-schlenkerl...
1,Duckpond Duckpond Darkwing De Luxe,3.8,https://www.ratebeer.com/beer/duckpond-duckpon...
2,Aecht Schlenkerla Weichsel Rotbier,3.8,https://www.ratebeer.com/beer/aecht-schlenkerl...
3,Ayinger Altbairisch Dunkel,3.6,https://www.ratebeer.com/beer/ayinger-altbairi...
4,Russian River Beatification (Batch 002+),4.2,https://www.ratebeer.com/beer/russian-river-be...


In [11]:
# Configure Chrome options for headless mode
chrome_options = Options()
chrome_options.binary_location = chrome_path  # Path to your Chrome browser
chrome_options.add_argument('--headless')  # Enable headless mode to run faster
chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration (for better performance in headless mode)
chrome_options.add_argument('--no-sandbox')  # Added for safe execution in certain environments
chrome_options.add_argument('--disable-dev-shm-usage')  # Avoid issues with shared memory

# Set up the Chrome WebDriver service
service = Service(driver_path)  # Path to your ChromeDriver

# Create a new instance of the Chrome driver
driver = webdriver.Chrome(service=service, options=chrome_options)

# Function to click "Show More" buttons on the current page
def click_show_more_buttons():
    # Locate all "Show More" buttons on the page
    show_more_xpath = '//span[contains(@class, "MuiButton-label") and text()="Show more"]'
    show_more_buttons = driver.find_elements(By.XPATH, show_more_xpath)

    # Click each "Show More" button
    for button in show_more_buttons:
        try:
            if button.is_displayed():
                driver.execute_script("arguments[0].click();", button)
                time.sleep(1)  # Wait for the content to expand
        except:
            pass  # Skip if there's an issue clicking the button

# Function to scrape reviews on the current page
def scrape_reviews():
    reviews = []
    ratings_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "bRPQdN", " " )) and contains(concat( " ", @class, " " ), concat( " ", "MuiTypography-subtitle1", " " ))]'  # Adjust this based on your page's structure for ratings
    messages_xpath = '//*[contains(concat( " ", @class, " " ), concat( " ", "pre-wrap", " " )) and contains(concat( " ", @class, " " ), concat( " ", "MuiTypography-body1", " " ))]'  # Adjust this based on your page's structure for review messages

    # Find all the review ratings and messages on the current page
    ratings = driver.find_elements(By.XPATH, ratings_xpath)
    messages = driver.find_elements(By.XPATH, messages_xpath)

    for i, message in enumerate(messages):
        reviews.append({
            'rating': ratings[i].text if i < len(ratings) else None,  # Handle index if ratings and messages mismatch
            'message': message.text
        })

    return reviews

# Function to scrape reviews for a single beer URL with a limit of 250 reviews
def scrape_beer_reviews(beer_name, url, review_limit):
    all_reviews = []
    
    driver.get(url)

    # Handle the cookies banner by accepting it
    try:
        accept_cookies_id = 'onetrust-accept-btn-handler'
        accept_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, accept_cookies_id)))
        accept_button.click()
    except:
        print("No cookies banner found or failed to dismiss.")
        pass

    # Loop through pages and scrape reviews until the limit is reached
    while len(all_reviews) < review_limit:
        # Click all "Show More" buttons to expand reviews
        click_show_more_buttons()

        # Scrape reviews on the current page
        page_reviews = scrape_reviews()
        all_reviews.extend(page_reviews)

        # Check if we've hit the review limit
        if len(all_reviews) >= review_limit:
            all_reviews = all_reviews[:review_limit]  # Truncate to exactly the review limit
            break

        # Find the "Next" button and move to the next page
        try:
            next_button_xpath = '//button[@aria-label="Next page" and contains(@class, "MuiIconButton-root")]'
            next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, next_button_xpath)))
            next_button.click()

            # Wait for the next page to load
            time.sleep(2)
        except:
            # If there's no "Next" button, break the loop
            print(f"Finished scraping {beer_name}.")
            break

    return all_reviews

# Initialize an empty DataFrame to store all the reviews
df = pd.DataFrame(columns=['Beer Name', 'URL', 'Rating', 'Review'])

# Iterate through each beer in the cleaned_beers list
for beer in cleaned_beers:
    beer_name = beer['name']
    beer_url = beer['link']

    print(f"Scraping reviews for {beer_name} at {beer_url}...")

    # Scrape reviews for the current beer with a limit of 250 reviews
    reviews = scrape_beer_reviews(beer_name, beer_url, review_limit=100)

    # Create a DataFrame for the reviews of the current beer
    beer_df = pd.DataFrame(reviews)
    beer_df['Beer Name'] = beer_name
    beer_df['URL'] = beer_url

    # Append the DataFrame for this beer to the overall DataFrame
    df = pd.concat([df, beer_df], ignore_index=True)

# Save the DataFrame to a CSV file
df.to_csv('expanded_beer_reviews.csv', index=False, encoding='utf-8')

# Close the browser after scraping all URLs
driver.quit()

print("Scraping completed. Data saved to 'expanded_beer_reviews.csv'")


Scraping reviews for Aecht Schlenkerla Rauchbier Urbock at https://www.ratebeer.com/beer/aecht-schlenkerla-rauchbier-urbock/6349/359461/...
Scraping reviews for Duckpond Duckpond Darkwing De Luxe at https://www.ratebeer.com/beer/duckpond-duckpond-darkwing-de-luxe/1039111/271988/...
No cookies banner found or failed to dismiss.
Finished scraping Duckpond Duckpond Darkwing De Luxe.
Scraping reviews for Aecht Schlenkerla Weichsel Rotbier at https://www.ratebeer.com/beer/aecht-schlenkerla-weichsel-rotbier/1120814/358150/...
No cookies banner found or failed to dismiss.


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=129.0.6668.90)
Stacktrace:
0   chromedriver                        0x0000000102d5d208 cxxbridge1$str$ptr + 1927396
1   chromedriver                        0x0000000102d5566c cxxbridge1$str$ptr + 1895752
2   chromedriver                        0x0000000102950808 cxxbridge1$string$len + 89564
3   chromedriver                        0x000000010293a658 core::str::slice_error_fail::h6c488016ada29016 + 64012
4   chromedriver                        0x000000010293a598 core::str::slice_error_fail::h6c488016ada29016 + 63820
5   chromedriver                        0x00000001029cdcac cxxbridge1$string$len + 602752
6   chromedriver                        0x0000000102989698 cxxbridge1$string$len + 322668
7   chromedriver                        0x000000010298a310 cxxbridge1$string$len + 325860
8   chromedriver                   